In this competition, our aim is to predict which LLM responses users will prefer in a head-to-head battle between chatbots powered by large language models (LLMs).

#Import Libraries

In [ ]:
!pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Suc

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras_nlp
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

In [ ]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

TensorFlow: 2.16.1
Keras: 3.3.3
KerasNLP: 0.12.1


In [ ]:
#CONFIGURATION SECTION
class CFG:
  seed = 42
  preset = "deberta_v3_extra_small_en"
  sequence_lenght = 512
  epochs = 3
  batch_seize = 24
  scheduler = 'cosine' # rate scheduler
  label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
  name2label = {v:k for k, v in label2name.items()}
  class_labels = list(label2name.keys())
  class_names = list(label2name.values())

In [ ]:
# Random seed for similar result in each run
keras.utils.set_random_seed(CFG.seed)

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16") #No float32, reducing GPU usage.

###Train Data

In [ ]:
df = pd.read_csv('/content/train.csv')
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])

df["class_name"] = df[["winner_model_a", "winner_model_b", "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)
df.head()

ParserError: Error tokenizing data. C error: EOF inside string starting at row 14545

###Test Data

In [ ]:
test_df = pd.read_csv('/content/test.csv')
# test_test_df = pd.read_csv(f'{BASE_PATH}/test.csv')
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

test_df.head()

,id,prompt,response_a,response_b
0,136060,"I have three oranges today, I ate an orange ye...",You have two oranges today.,You still have three oranges. Eating an orange...
1,211333,You are a mediator in a heated political debat...,Thank you for sharing the details of the situa...,Mr Reddy and Ms Blue both have valid points in...
2,1233961,How to initialize the classification head when...,When you want to initialize the classification...,To initialize the classification head when per...


##Contextualize Response with Prompt

In [ ]:
def make_pairs(row):
  row["encode_fail"] = False
  try:
    prompt = row.prompt.encode("utf-8").decode("utf-8")
  except:
    prompt = ""
    row["encode_fail"] = True
  try:
    response_a = row.response_a.encode("utf-8").decode("utf-8")
  except:
    response_a = ""
    row["encode_fail"] = True
  try:
    response_b = row.response_b.encode("utf-8").decode("utf-8")
  except:
    response_b = ""
    row["encode_fail"] = True
  row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",
                    f"Prompt: {prompt}\n\nResponse: {response_b}"]
  return row

In [ ]:
df = df.apply(make_pairs, axis=1)
display(df.head(2))
test_df = test_df.apply(make_pairs, axis=1)
display(test_df.head(2))

NameError: name 'df' is not defined

##Encoding Fail Statistics
> 1% of the samples failed to be encoded, while  99% of the samples don't have any issues.

In [ ]:
df.encode_fail.value_counts(normalize=False)

NameError: name 'df' is not defined

In [ ]:
model_df = pd.concat([df.model_a, df.model_b])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Create a bar plot with custom styling using Plotly
fig = px.bar(counts, x='LLM', y='Count',
             title='Distribution of LLMs',
             color='Count', color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

NameError: name 'df' is not defined

Winning Distribution model


In [ ]:
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

fig = px.bar(counts, x='Winner', y='Win Count',
             title='Winner distribution for Train Data',
             labels={'Winner': 'Winner', 'Win Count': 'Win Count'},
             color='Winner', color_continuous_scale='viridis')

fig.update_layout(xaxis_title="Winner", yaxis_title="Win Count")

fig.show()

NameError: name 'df' is not defined

###Data split

In [ ]:
from sklearn.model_selection import train_test_split
trian_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

NameError: name 'df' is not defined

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

AttributeError: type object 'CFG' has no attribute 'sequence_length'

In [ ]:
outs = preprocessor(df.options.iloc[0])
for k, v in outs.items():
  print(k, ":", v.shape)

NameError: name 'preprocessor' is not defined

### Dataloader

In [ ]:
def build_dataset(texts, labels=None, batch_size=32, cache =True, shuffle=1124):
  AUTO = tf.data.AUTOTUNE
  slices = (texts,) if labels is None else(texts, keras.utils.to_categorical(labels,num_classes=3))
  ds = tf.data.Dataset.from_tensor_slices(slices)
  ds = ds.cache() if cache else ds
  ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)
  opt = tf.data.Options()
  if shuffle:
    ds = ds.shuffle(shuffle, seed=CFG.seed)
    opt.experimental_deterministic = False
  ds = ds.with_options(opt)
  ds = ds.batch(batch_size, drop_remainder=False)
  ds = ds.prefetch(AUTO)
  return ds

#Build & Train

In [ ]:
train_texts = train_df.options.tolist()
train_labels = train_df.class_label.tolist()
trian_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)
# Validation part
valid_texts = valid_df.options.tolist()
valid_labels = valid_df.class_label.tolist()

valid_df = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)

NameError: name 'train_df' is not defined

###Schedule - transfer learning

In [ ]:
import math
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [ ]:
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

AttributeError: type object 'CFG' has no attribute 'batch_size'

##Model Checkpointing

In [ ]:
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',
                                          save_best_only=True,
                                          save_weights_only=True,
                                          mode='min')  # Get Model checkpoint callback

In [ ]:
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

#Creating MODEL

In [ ]:
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# Create a DebertaV3Classifier backbone
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# Compute embeddings for first response: (P + R_A) using backbone
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# Compute embeddings for second response: (P + R_B), using the same backbone
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# Compute final output
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

100%|██████████| 539/539 [00:00<00:00, 98.0kB/s]
100%|██████████| 270M/270M [00:08<00:00, 34.9MB/s]


NameError: name 'log_loss' is not defined

In [ ]:
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

##Training phase

In [ ]:
history = model.fit(
    trian_ds,
    epochs = CFG.epochs,
    vlaidation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)

NameError: name 'trian_ds' is not defined

##Loading Best Model

In [ ]:
model.load_weights('/best_model.h5')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

###Making prediction based on model

In [ ]:
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)

In [ ]:
test_preds = model.preds(test_ds, verbose=1)

AttributeError: 'Functional' object has no attribute 'preds'

###Submission

In [ ]:
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

NameError: name 'test_preds' is not defined

##Preparation Submission File

In [ ]:
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()